In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-27 00:47:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.48MB/s    in 0.2s    

2023-02-27 00:47:44 (5.48 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://big-data-etl.s3.us-east-2.amazonaws.com/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz" 

spark.sparkContext.addFile(url)
user_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)


# Show DataFrame
user_reviews_df.show(50)


+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
# Get the number of rows in the DataFrame.
print(user_reviews_df.count())

904765


In [6]:
vine_reviews_df = user_reviews_df.filter(user_reviews_df.vine == 'Y')
vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25575022| RCRKN2GJFD3ID|B00S2VVTP4|     489028362|Ernie Ball Regula...|Musical Instruments|          5|            0|          0|   Y|                N|Smooth Sound Quality|Smooth sound qual...| 2015-08-31|
|         US|   52752518|R1IVB6FXFEKM6X|B00ZU4G0ZK|     499223759|TC Electronic Pol...|Musical Instruments| 

In [7]:
print(vine_reviews_df.count())

2287


In [8]:
vine_reviews_df = vine_reviews_df.dropna()

In [9]:
print(vine_reviews_df.count())

2287


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [11]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [12]:
tokenizer = Tokenizer(inputCol="review_body", outputCol="words")
tokenizer

Tokenizer_cab5b2528a9f

In [13]:
def word_list_length(word_list):
    return len(word_list)

In [14]:
count_tokens = udf(word_list_length, IntegerType())
count_tokens

<function __main__.word_list_length(word_list)>

In [15]:
# Transform DataFrame
vine_reviews_df = tokenizer.transform(vine_reviews_df)
vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   25575022| RCRKN2GJFD3ID|B00S2VVTP4|     489028362|Ernie Ball Regula...|Musical Instruments|          5|            0|          0|   Y|                N|Smooth Sound Quality|Smooth sound qual...| 2015-08-31|[smooth, sound, q...|
|         US|   52752518

In [16]:

# Select the needed columns and don't truncate results
# vine_reviews_df.select("review_headline", "words")
vine_reviews_df = vine_reviews_df.withColumn("tokens", count_tokens(col("words")))
vine_reviews_df.show()





+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|tokens|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|         US|   25575022| RCRKN2GJFD3ID|B00S2VVTP4|     489028362|Ernie Ball Regula...|Musical Instruments|          5|            0|          0|   Y|                N|Smooth Sound Quality|Smooth sound qual...| 2015-08-31|[smooth, sound, q...|    

In [17]:
vine_reviews_df = vine_reviews_df.filter(vine_reviews_df.tokens >10)
print(vine_reviews_df.count())

2284


In [18]:
vine_reviews_df.filter(vine_reviews_df.tokens < 20).show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|tokens|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|         US|   46877221|R13JRKZHUTADKJ|B007KC7IAG|     120573562|Ernie Ball Slinky...|Musical Instruments|          5|            0|          0|   Y|                N|Highly recommende...|These produce a g...| 2015-08-13|[these, produce, ...|    

In [19]:
vine_helpful_votes_df = vine_reviews_df.filter(vine_reviews_df.helpful_votes != 0)
vine_helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|tokens|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|         US|   27769608|R1MESKI5XK08OC|B00YRKPB3K|     219663516|D'Addario Reserve...|Musical Instruments|          5|            2|          3|   Y|                N|A good reed, that...|Clarinet and saxo...| 2015-08-31|[clarinet, and, s...|    

In [20]:
print(vine_helpful_votes_df.count())

1062


In [21]:
vine_total_votes_df = vine_reviews_df.filter(vine_reviews_df.total_votes != 0)
vine_total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|tokens|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|         US|   27769608|R1MESKI5XK08OC|B00YRKPB3K|     219663516|D'Addario Reserve...|Musical Instruments|          5|            2|          3|   Y|                N|A good reed, that...|Clarinet and saxo...| 2015-08-31|[clarinet, and, s...|    

In [22]:
print(vine_total_votes_df.count())

1343


In [23]:
non_vine_reviews_df = user_reviews_df.filter(user_reviews_df.vine == 'N')
non_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [24]:
print(non_vine_reviews_df.count())

902476


In [25]:
non_vine_reviews_df = non_vine_reviews_df.dropna()

In [26]:
print(non_vine_reviews_df.count())

902376


In [27]:
non_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [28]:
non_vine_reviews_df = tokenizer.transform(non_vine_reviews_df)
non_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|[works, very, goo...|
|         US|   14640079

In [29]:
non_vine_reviews_df = non_vine_reviews_df.withColumn("tokens", count_tokens(col("words")))
non_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|               words|tokens|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|[works, very, goo...|    

In [30]:
non_vine_reviews = non_vine_reviews_df.filter(non_vine_reviews_df.tokens > 10)
print(non_vine_reviews.count())

744572


In [31]:
non_vine_helpful_votes = non_vine_reviews_df.filter(non_vine_reviews_df.helpful_votes != 0)
print(non_vine_helpful_votes.count())

351209


In [32]:
non_vine_total_votes = non_vine_reviews_df.filter(non_vine_reviews_df.total_votes != 0)
print(non_vine_total_votes.count())

419987


In [36]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-etl-db.cxzqhrhgawdk.us-east-2.rds.amazonaws.com:5432/big-data-etl-db"
config = {"user":"root", "password": "42517131", "driver":"org.postgresql.Driver"}

In [37]:
# Write customers_df to table in RDS
vine_reviews_df.write.jdbc(url=jdbc_url, table='vine_reviews', mode=mode, properties=config)